In [1]:
import pandas as pd
from vncorenlp import VnCoreNLP
import ast

In [13]:
train_df = pd.read_csv('./data/train_data.csv').drop('Unnamed: 0', axis=1)
test_df = pd.read_csv('./data/test_data.csv').drop('Unnamed: 0', axis=1)
# train_df.head()
test_df.head()

,NewsTitle,NewsFullContent,label
0,MWG lãi gần 900 tỷ đồng sau 5 tháng,Về cơ cấu doanh thu doanh thu từ chuỗi bán lẻ ...,"[('về', 1), ('cơ cấu', 0), ('doanh thu', 1), (..."
1,"6 tháng, PNJ báo lãi hợp nhất đạt hơn 146 tỷ đồng",Theo đó doanh thu thuần trong quý 2 của PNJ ...,"[('theo', 1), ('đó', 0), ('doanh thu', 1), ('t..."
2,"2 tháng sau khi lên làm CEO, ông Trần Kinh Doa...",Ông Trần Kinh Doanh Tổng giám đốc Công ty cổ p...,"[('ông', 0), ('trần kinh doanh tổng giám đốc',..."
3,BIDV rao bán tàu Ocean Queen với giá khởi điểm...,Giá khởi điểm hơn 300 tỷ đồng nói trên không b...,"[('giá', 1), ('khởi điểm', 0), ('hơn', 1), ('3..."
4,"PNJ trích dự phòng thêm 115 tỷ đồng, đặt kế ho...",Đồng thời PNJ dự kiến chỉ tiêu lợi nhuận 361...,"[('đồng thời', 1), ('pnj', 1), ('dự kiến', 1),..."


# NER feature

In [14]:
annotator = VnCoreNLP(r"./VnCoreNLP-master/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [15]:
def word_segment(text):
    annotated_text = annotator.annotate(text)['sentences'][0]
    return [dic['form'].replace('_',' ').lower() for dic in annotated_text] 

In [16]:
train_df['word_segment'] = train_df['NewsFullContent'].apply(lambda text: word_segment(text))
test_df['word_segment'] = test_df['NewsFullContent'].apply(lambda text: word_segment(text))

In [17]:
ner = VnCoreNLP("./VnCoreNLP-master/VnCoreNLP-1.1.1.jar", annotators="wseg,pos,ner", max_heap_size='-Xmx2g')

In [18]:
def vnner(text):
    annotated_text = ner.annotate(text)
    pos = []
    for dic in annotated_text['sentences'][0]:
        if dic['nerLabel'] != 'O':
            pos.append(dic['form'].replace('_',' ').lower())
        
    return list(set(pos))

def tag(word, dic):
    if word in dic:
        return True
    else:
        return False

def ner_tag(row):
    keywords = vnner(row['NewsFullContent'])
    text = row['word_segment']
    text_df = pd.DataFrame(text)
    text_df.columns = ['word']

    text_df['tag'] = text_df['word'].apply(lambda x: tag(x, keywords))
    
    # return list(zip(text_df['word'], text_df['tag']))
    return list(text_df['tag'])

In [19]:
name_entity_reg = []
for i,row in train_df.iterrows():
    name_entity_reg.append(ner_tag(row))

train_df['name_entity'] = name_entity_reg

In [20]:
name_entity_reg = []
for i,row in test_df.iterrows():
    name_entity_reg.append(ner_tag(row))

test_df['name_entity'] = name_entity_reg

# Title

In [21]:
def tag(word, dic):
    if word in dic:
        return True
    else:
        return False

def title_detect(row):
    annotated_text = annotator.annotate(row['NewsTitle'])['sentences'][0]
    title = [dic['form'].replace('_',' ').lower() for dic in annotated_text] 
    
    text = row['word_segment']
    text_df = pd.DataFrame(text)
    text_df.columns = ['word']

    text_df['tag'] = text_df['word'].apply(lambda x: tag(x, title))
    
    return list(text_df['tag'])

In [24]:
title_tag = []
for i,row in train_df.iterrows():
    title_tag.append(title_detect(row))

train_df['title_tag'] = title_tag

In [25]:
title_tag = []
for i,row in test_df.iterrows():
    title_tag.append(title_detect(row))

test_df['title_tag'] = title_tag

In [28]:
train_df[['name_entity', 'title_tag']].to_csv('./data/training_feature.csv')
test_df[['name_entity', 'title_tag']].to_csv('./data/testing_feature.csv')

In [30]:
newtest = pd.read_csv('./data/newtest.csv').drop('Unnamed: 0', axis=1)
newtest['word_segment'] = newtest['NewsFullContent'].apply(lambda text: word_segment(text))

# ner feature
name_entity_reg = []
for i,row in newtest.iterrows():
    name_entity_reg.append(ner_tag(row))

newtest['name_entity'] = name_entity_reg
2
# istitile feature
title_tag = []
for i,row in newtest.iterrows():
    title_tag.append(title_detect(row))

newtest['title_tag'] = title_tag

In [35]:
newtest[['name_entity', 'title_tag']].to_csv('./data/newtest_feature.csv')